## Primer

Consider the class `dummy` below.

In [53]:
class dummy:
    def __init__(self, name):
        self.name = name

We start off with assigning two variables `a` and `b` to two `dummy` objects with their attribute `name` to be $1$ and $2$ respectively.

Then we set a new variable `c` to be equals to `a`, we call `c` a **reference** to `a` and they are of the same object referenced to `dummy(name=1)` as the assertion holds.

Now at **line 9**, we re-assign `c` to `b`, and we assert that both `a` and `c` are different objects now. Main point is that the variable `a` is untouched, remaining $1$ the whole time.

> We say that `c` is merely a reference to `a` and if we assign a new value (variable) to `c`, you don't change `a`.

In [57]:
a = dummy(name = 1)
b = dummy(name = 2)
print(f"a={a.name}, b={b.name}")

c = a
print(f"a={a.name}, c={c.name}")
assert id(a) == id(c)

c = b
print(f"a={a.name}, c={c.name}")
assert id(a) != id(c)

a=1, b=2
a=1, c=1
a=1, c=2


The setup is exactly the same as previous up till **line 7**. Instead of assigning `c` to another variable directly, we first change the attribute (property) of the object `c` to become $3$ by `c.name = 3`. 

Now if we print `a.name` and `c.name` again, we magically find that `a.name` is changed from $1$ to $3$, moreover, both `a` and `c` are still referencing to the same object.

> We conclude that if you access a member (attribute) of the reference variable `c`, you're actually accessing a member (attribute) of `a`. So, if you assign a new value to the member (attribute) of `c`, you're assigning the same value to that member (attribute) of `a`.

In [60]:
a = dummy(name = 1)
b = dummy(name = 2)
print(f"a={a.name}, b={b.name}")

c = a
print(f"a={a.name}, c={c.name}")
assert id(a) == id(c)

c.name = 3
print(f"a={a.name}, c={c.name}")
assert id(a) == id(c)

a=1, b=2
a=1, c=1
a=3, c=3


## Confusion 1

In [5]:
from typing import *

class Node:
    curr_node_value: Any
    # next_node: Node
    def __init__(self, curr_node_value: Any = None):
        self.curr_node_value = curr_node_value
        self.next_node = None
        
class LinkedList:
    def __init__(self):
        self.head = None
        
        
ll1_first = Node(1)
ll1_second = Node(2)
ll1_third = Node(3)

ll2_first = Node(4)
ll2_second = Node(5)
ll2_third = Node(6)

# create llist 1
ll1 = LinkedList()
ll1.head = ll1_first
ll1.head.next_node = ll1_second
ll1.head.next_node.next_node = ll1_third

# create llist 2
ll2 = LinkedList()
ll2.head = ll2_first
ll2.head.next_node = ll2_second
ll2.head.next_node.next_node = ll2_third


merged_sorted_llist = LinkedList()

ll1_temp_curr_node = ll1.head
ll2_temp_curr_node = ll2.head

while ll1_temp_curr_node is not None and ll2_temp_curr_node is not None:
    print(ll1_temp_curr_node.curr_node_value)
    ll1_curr_node = ll1_temp_curr_node
    ll2_curr_node = ll2_temp_curr_node
 
    if ll1_curr_node.curr_node_value <= ll2_curr_node.curr_node_value:
        merged_sorted_llist.head = ll1_curr_node
        merged_sorted_llist.head.next_node = ll2_curr_node
 

    ll1_temp_curr_node = ll1_temp_curr_node.next_node
    ll2_temp_curr_node = ll2_temp_curr_node.next_node

1
4
5


### Scenario 1

If you comment out **line 48**, then **line 42**'s print statement `print(ll1_temp_curr_node.curr_node_value)` will give you the following output:

```python
print(ll1_temp_curr_node.curr_node_value)print(ll1_temp_curr_node.curr_node_value)
1
2
3
```

Intuitively, this should be the case since our `ll1_temp_curr_node` starts off with `ll1.head`, and at **line 51**, we re-assign `ll1_temp_curr_node` to its next node `ll1_temp_curr_node.next_node` so that in our next iteration it prints out the node value $2$, and so on.

The scenario gets out of hand soon if you uncomment **line 42**.

### Scenario 2

If you uncomment out **line 48**, then **line 42**'s print statement `print(ll1_temp_curr_node.curr_node_value)` will give you the following output:

```python
print(ll1_temp_curr_node.curr_node_value)print(ll1_temp_curr_node.curr_node_value)
1
4
5
```

This gets confusing since I did not **explicitly made any changes besides adding a harmless looking line**. Why then did in the second iteration, `ll1_temp_curr_node.curr_node_value` gives us $4$ instead of $2$? This has to originate from the logic we laid out in the section **Primer**. The logic goes:

```python
ll1_temp_curr_node = ll1.head # ll1_temp_curr_node is a reference to ll1.head
ll1_curr_node = ll1_temp_curr_node # ll1_curr_node is a reference to ll1_temp_curr_node
merged_sorted_llist.head = ll1_curr_node # this part is ok but main thing is to remember that merged_sorted_llist.head is a reference to ll1_curr_node
merged_sorted_llist.head.next_node = ll2_curr_node # this part is where things go tricky, we explain more below.
ll1_temp_curr_node = ll1_temp_curr_node.next_node # we explain more below
```

The second last line is problematic, we made a change to the attribute `next_node` of `merged_sorted_llist.head` and so our `merged_sorted_llist.head.next_node` holds a value of $4$ (the current value of `ll2_curr_node`), and by our logic in **Primer**, this changes the attribute `next_node` in `ll1_curr_node`, so now `ll1_curr_node`'s `next_node` holds a value of $4$; and recall that `ll1_curr_node` is also a reference to `ll1_temp_curr_node`, so changing the attribute of `ll1_curr_node` changes the attribute of `ll1_temp_curr_node`, so that `ll1_temp_curr_node` has a `next_node` of $4$. 

To recap, a small change in **line 48** changes the next node of `ll1_temp_curr_node` and that is why when you call **line 51**, our poor `ll1_temp_curr_node` did go to the next node, but the next node is no longer $2$ and is now $4$.

## Confusion 2

So the name of the game is that:

> Using **Primer** as an example: in most cases in Python, when you perform an assignment on a variable, `c` in this case, the value of `c` changes, but the object `a` that it was originally referring to does not. This is because Python variables are simply references/pointers to objects. However, when you perform an assignment such that the reference variable changes its properties (attribute), then the original referenced object will change!

However, when calling the `reverse()` method in a Linked List, one would think that **line 43** changing `temp`'s attribute will affect the value of `curr_node`.

The original logic was that:

- We assigned `temp` as a reference to `curr_node` in **line 36**.
- We changed the attribute of the reference variable `temp` by setting its `next_node` to `prev_node`. Thus, `curr_node`'s `next_node` should also be `prev_node` (which is `None`). This is not the case, why?

The reason is simple, at **line 40**, we **overwrite** `curr_node` to be something else, at this point in time, `temp` will lose its connection to `curr_node`, and is no longer a reference to it. Thus, when we did something to its attribute in **line 43**, `curr_node` is unchanged.

In [75]:
from typing import *

class Node:
    curr_node_value: Any
    # next_node: Node
    def __init__(self, curr_node_value: Any = None):
        self.curr_node_value = curr_node_value
        self.next_node = None
        
class LinkedList:
    def __init__(self):
        self.head = None

    @staticmethod
    def traverse(head_node: Node):
        # stay true to the idea of having None as the "last last Node"
        temp = head_node
        
        while temp is not None:
            print(temp.curr_node_value)
            temp = temp.next_node
            if temp is None:
                print("None")
                
    @classmethod
    def reverse(cls, head_node: Node):
        print("Traverse current head node:")
        cls.traverse(head_node)
        
        prev_node = None
        curr_node = head_node
        # print(id(head_node), id(curr_node))
            
        counter = 1
        while curr_node is not None:
                temp = curr_node
                # print(id(temp), id(curr_node))
                print(f"loop {counter}: curr_node={curr_node.curr_node_value}")

                curr_node = curr_node.next_node
                # print(id(temp), id(curr_node))

                temp.next_node = prev_node
                # print(id(temp), id(curr_node))
                print(f"loop {counter}: curr_node={curr_node.curr_node_value}")

                prev_node = temp

                counter += 1
        
        reversed_head_node = prev_node
        
        print("Traverse reversed head node:")
        cls.traverse(reversed_head_node)
        return reversed_head_node
        
ll1_first = Node(1)
ll1_second = Node(2)
ll1_third = Node(3)

# create llist 1
ll1 = LinkedList()
ll1.head = ll1_first
ll1.head.next_node = ll1_second
ll1.head.next_node.next_node = ll1_third


_ = ll1.reverse(ll1.head)

Traverse current head node:
1
2
3
None
loop 1: curr_node=1
loop 1: curr_node=2
loop 2: curr_node=2
loop 2: curr_node=3
loop 3: curr_node=3


AttributeError: 'NoneType' object has no attribute 'curr_node_value'

## Reference

- https://stackoverflow.com/questions/58759348/when-does-a-pointer-to-a-linked-list-change-the-actual-list